# Introduction
We'll be applying naive bayes to determine whether a tweet was written by [Donald Trump](https://twitter.com/realDonaldTrump) or [Hillary Clinton](https://twitter.com/HillaryClinton). For a more in-depth theoretical explanation of the theory behind the classifier, see [our notes](https://github.com/MachinesWhoLearn/lectures/blob/master/2016-2017.Meetings/05.DIY_naive_bayes/naive_bayes_primer/naive_bayes_primer.pdf).

### Features
In our model, the features are the individual words. For example, we'd expect that the word "wall" would be more likely to appear in Trump tweets, so we want to account for that in our model. To start off with, we'll load the text data and then perform some basic tokenization to separate out all the words.

# Tokenizing and Counting the Words
We want to transform our collection of tweets into something that the model can understand. A basic idea from natural language processing (NLP) is the bag of words approach (BOW). When we use the bag of words, we simply count the number of times a word occurs in a document, and divide it by the total number of words. Doing this for each word, we can get a probability distribution for the probability of a word occurring.

Let's start by loading our training data (the raw tweets) and the associated labels (indicating authorship, "0" for Hillary and "1" for Trump). We'll then count up all the statistics we'll need to use later for calculating probabilities

In [1]:
# setting up some imports and some miscellaneous helper methods we'll be using
import re
import string
from math import log
from __future__ import print_function

# our default smoothing value, we pretend we see each word at
# least once so all our probabilities are well-formed 
# (so no multiplying by 0 if a word isn't in train set)
SMOOTHING = 1.0

# location of data file relative to this notebook path
TRAIN_DATA_PATH = "../data/tweets_train.txt"
TRAIN_LABELS_PATH = "../data/labels_train.txt"
TEST_DATA_PATH = "../data/tweets_test.txt"
TEST_LABELS_PATH = "../data/labels_test.txt"

# strip punctuation from a string
def remove_punctuation(input_str):
    "from http://stackoverflow.com/questions/265960/best-way-to-strip-punctuation-from-a-string-in-python"
    table = string.maketrans("","")
    return input_str.translate(table, string.punctuation)

# split a line into tokens, or a list of words (known as tokens) 
# that are separated by whitespace
def tokenize(line):
    line = remove_punctuation(line)
    line = line.lower().strip()
    # re.split has an odd tendency to add empty strings, remove those
    return [token for token in re.split("\W+", line) if token != '']
 
# given a string and a dictionary with counts, 
# count the frequency of words in the string and 
# tabulate them in the dictionary "counts".
def count_words(tokens):
    counts = {}
    for token in tokens:
        if token in counts:
            counts[token] += 1.0
        else:
            counts[token] = 1.0
    return counts

In [2]:
# Setting up some the data structures we'll use to keep track of counts

# this dictionary maps the numeric label to the candidate name
# the "\n" after each label accounts for the newline character 
# that follows each one
LABEL_MAP = {'0\n': 'hillary', '1\n': 'trump'}

# this dictionary keeps track of the number of time we have seen a word
# for each candidate
word_counts_per_candidate = {
    "trump": {},
    "hillary": {}
}
# this dictionary keeps track of how many trump tweets there are and how many
# hillary tweets there are. These are our priors!
total_tweet_count = {'trump': 0.0, 'hillary': 0.0}

# counts words across all tweets
vocabulary = {}

Let's take a moment to verify that `count_words` does what we want it to.

In [3]:
sample_tweet = "I am growing the Republican Party tremendously - just look at the numbers, way up! Democrats numbers are significantly down from years past."
sample_tokens = tokenize(sample_tweet)
count_words(sample_tokens)

{'am': 1.0,
 'are': 1.0,
 'at': 1.0,
 'democrats': 1.0,
 'down': 1.0,
 'from': 1.0,
 'growing': 1.0,
 'i': 1.0,
 'just': 1.0,
 'look': 1.0,
 'numbers': 2.0,
 'party': 1.0,
 'past': 1.0,
 'republican': 1.0,
 'significantly': 1.0,
 'the': 2.0,
 'tremendously': 1.0,
 'up': 1.0,
 'way': 1.0,
 'years': 1.0}

In [4]:
# open the file with train data and refer to it as "data", and
# open the file with train labels and refer to it as "labels"
with open(TRAIN_DATA_PATH, 'r') as data, open(TRAIN_LABELS_PATH, 'r') as labels:
    # zip data and labels together to create a list of tuples of (tweet,label),
    # which we then iterate over. the tweet variable refers to an individual tweet
    # and the label variable refers to an individual label
    for tweet, label in zip(data, labels):
        # if it is a Hillary tweet, increment the "hillary" element of `total_tweet_count`
        # if it is a Trump tweet, increment the "trump" element of `total_tweet_count`.
        total_tweet_count[LABEL_MAP[label]] += 1.0
        
        tokens = tokenize(tweet)
        # get the word counts from the tweet
        word_counts = count_words(tokens)
        
        # iterate over the words we found in the tweet
        for word, count in word_counts.items():
            # if we haven't seen a word yet, let's add it to our dictionary with a 
            # count of 2 * how much we smoothing up by (`SMOOTHING`),
            # (since we presume that both hillary and trump have seen it)
            # we'll be incrementing it later
            if word not in vocabulary:
                vocabulary[word] = 2 * SMOOTHING
                # similarily, add it to each candidate's individual dictionary
                word_counts_per_candidate["trump"][word] = SMOOTHING
                word_counts_per_candidate["hillary"][word] = SMOOTHING
            # now add the number of times we saw the word
            # to the global dictionary and the associated candidate's 
            # dictionary
            vocabulary[word] += count
            word_counts_per_candidate[LABEL_MAP[label]][word] += count

Let's take a look at our output

In [5]:
total_tweet_count

{'hillary': 2404.0, 'trump': 2362.0}

In [6]:
vocabulary

{'foul': 3.0,
 'four': 27.0,
 'rawlingsblake': 3.0,
 'railing': 4.0,
 'plaudits': 3.0,
 'looking': 37.0,
 'eligible': 3.0,
 'electricity': 3.0,
 'igual': 3.0,
 'lord': 3.0,
 'instate': 3.0,
 'dels': 3.0,
 'unify': 4.0,
 'broward': 3.0,
 'bringing': 9.0,
 'wednesday': 12.0,
 'romneys': 4.0,
 'infringed': 3.0,
 'tired': 5.0,
 'pulse': 3.0,
 '11pme': 3.0,
 'second': 10.0,
 '278': 3.0,
 'brexit': 9.0,
 'battleground': 5.0,
 'thunder': 7.0,
 'contributed': 4.0,
 'resilient': 5.0,
 'contributes': 3.0,
 'progrowth': 3.0,
 'hero': 5.0,
 'reporter': 11.0,
 'donalds': 5.0,
 'jasmine': 3.0,
 'springfield1pm': 3.0,
 'here': 93.0,
 'lgbt': 16.0,
 'china': 10.0,
 'kids': 49.0,
 'k': 3.0,
 'fouryear': 3.0,
 'reports': 5.0,
 'controversy': 3.0,
 'military': 30.0,
 'criticism': 5.0,
 'golden': 4.0,
 'divide': 6.0,
 'replace': 15.0,
 'brought': 13.0,
 'univ': 3.0,
 'unit': 3.0,
 'opponents': 6.0,
 'cheating': 4.0,
 'spoke': 11.0,
 'dnc': 11.0,
 'music': 3.0,
 'bedrock': 3.0,
 'until': 30.0,
 'paperwork'

In [7]:
word_counts_per_candidate["hillary"]

{'foul': 1.0,
 'four': 11.0,
 'rawlingsblake': 1.0,
 'railing': 2.0,
 'plaudits': 1.0,
 'looking': 7.0,
 'eligible': 2.0,
 'electricity': 1.0,
 'igual': 2.0,
 'lord': 1.0,
 'instate': 2.0,
 'dels': 1.0,
 'unify': 1.0,
 'broward': 2.0,
 'bringing': 4.0,
 'wednesday': 3.0,
 'romneys': 1.0,
 'infringed': 1.0,
 'tired': 2.0,
 'pulse': 2.0,
 '11pme': 1.0,
 'second': 5.0,
 '278': 1.0,
 'brexit': 1.0,
 'battleground': 3.0,
 'thunder': 1.0,
 'contributed': 3.0,
 'resilient': 4.0,
 'contributes': 2.0,
 'progrowth': 1.0,
 'hero': 2.0,
 'reporter': 3.0,
 'donalds': 3.0,
 'jasmine': 2.0,
 'springfield1pm': 1.0,
 'here': 65.0,
 'lgbt': 14.0,
 'china': 5.0,
 'kids': 46.0,
 'k': 2.0,
 'fouryear': 2.0,
 'reports': 2.0,
 'controversy': 1.0,
 'military': 20.0,
 'criticism': 3.0,
 'golden': 1.0,
 'divide': 5.0,
 'replace': 1.0,
 'brought': 6.0,
 'univ': 1.0,
 'unit': 2.0,
 'opponents': 1.0,
 'cheating': 2.0,
 'spoke': 3.0,
 'dnc': 5.0,
 'music': 1.0,
 'bedrock': 2.0,
 'until': 20.0,
 'paperwork': 2.0,
 '

In [8]:
word_counts_per_candidate["trump"]

{'foul': 2.0,
 'four': 16.0,
 'rawlingsblake': 2.0,
 'railing': 2.0,
 'plaudits': 2.0,
 'looking': 30.0,
 'eligible': 1.0,
 'electricity': 2.0,
 'igual': 1.0,
 'lord': 2.0,
 'instate': 1.0,
 'dels': 2.0,
 'unify': 3.0,
 'broward': 1.0,
 'bringing': 5.0,
 'wednesday': 9.0,
 'romneys': 3.0,
 'infringed': 2.0,
 'tired': 3.0,
 'pulse': 1.0,
 '11pme': 2.0,
 'second': 5.0,
 '278': 2.0,
 'brexit': 8.0,
 'battleground': 2.0,
 'thunder': 6.0,
 'contributed': 1.0,
 'resilient': 1.0,
 'contributes': 1.0,
 'progrowth': 2.0,
 'hero': 3.0,
 'reporter': 8.0,
 'donalds': 2.0,
 'jasmine': 1.0,
 'springfield1pm': 2.0,
 'here': 28.0,
 'lgbt': 2.0,
 'china': 5.0,
 'kids': 3.0,
 'k': 1.0,
 'fouryear': 1.0,
 'reports': 3.0,
 'controversy': 2.0,
 'military': 10.0,
 'criticism': 2.0,
 'golden': 3.0,
 'divide': 1.0,
 'replace': 14.0,
 'brought': 7.0,
 'univ': 2.0,
 'unit': 1.0,
 'opponents': 5.0,
 'cheating': 2.0,
 'spoke': 8.0,
 'dnc': 6.0,
 'music': 2.0,
 'bedrock': 1.0,
 'until': 10.0,
 'paperwork': 1.0,
 '

# Calculating Conditional Probabilities
Now, we'll proceed to calculate the the conditional probabilities for each word; that is, we want to know $\mathbb{P}(word|candidate)$, for each candidate. More verbosely, we will be calculating $\mathbb{P}(word|H)$ and $\mathbb{P}(word|T)$ for each in our vocabulary and for each event $H$ and $T$ (where $H$ is the event that Hillary Clinton is the author, and $T$ is the event that Donald Trump is the author).

In [9]:
word_probabilities_per_candidate = {
    "trump": {},
    "hillary": {}
}

# iterate over the hillary and trump dictionaries within 
# `word_count_per_candidate` with this outside loop
for candidate in word_counts_per_candidate:
    for word, count in word_counts_per_candidate[candidate].items():
        word_probabilities_per_candidate[candidate][word] = count / total_tweet_count[candidate]

Now let's take a look at our calculated probabilities

In [10]:
word_probabilities_per_candidate["hillary"]

{'foul': 0.00041597337770382697,
 'four': 0.004575707154742097,
 'railing': 0.0008319467554076539,
 'plaudits': 0.00041597337770382697,
 'looking': 0.0029118136439267887,
 'eligible': 0.0008319467554076539,
 'electricity': 0.00041597337770382697,
 'igual': 0.0008319467554076539,
 'lord': 0.00041597337770382697,
 'goodpaying': 0.0024958402662229617,
 'dels': 0.00041597337770382697,
 'unify': 0.00041597337770382697,
 'broward': 0.0008319467554076539,
 'bringing': 0.0016638935108153079,
 'wednesday': 0.0012479201331114808,
 'straight': 0.0024958402662229617,
 'infringed': 0.00041597337770382697,
 'tired': 0.0008319467554076539,
 'pulse': 0.0008319467554076539,
 'elections': 0.0020798668885191347,
 'second': 0.0020798668885191347,
 '278': 0.00041597337770382697,
 'brexit': 0.00041597337770382697,
 'captain': 0.0016638935108153079,
 'thunder': 0.00041597337770382697,
 'contributed': 0.0012479201331114808,
 'resilient': 0.0016638935108153079,
 'increasing': 0.00041597337770382697,
 'progrowt

In [11]:
word_probabilities_per_candidate["trump"]

{'foul': 0.000846740050804403,
 'four': 0.006773920406435224,
 'railing': 0.000846740050804403,
 'plaudits': 0.000846740050804403,
 'looking': 0.012701100762066046,
 'eligible': 0.0004233700254022015,
 'electricity': 0.000846740050804403,
 'igual': 0.0004233700254022015,
 'lord': 0.000846740050804403,
 'goodpaying': 0.0004233700254022015,
 'dels': 0.000846740050804403,
 'unify': 0.0012701100762066045,
 'broward': 0.0004233700254022015,
 'bringing': 0.0021168501270110076,
 'wednesday': 0.003810330228619814,
 'straight': 0.0021168501270110076,
 'infringed': 0.000846740050804403,
 'tired': 0.0012701100762066045,
 'pulse': 0.0004233700254022015,
 'elections': 0.0012701100762066045,
 'second': 0.0021168501270110076,
 '278': 0.000846740050804403,
 'brexit': 0.003386960203217612,
 'captain': 0.000846740050804403,
 'thunder': 0.002540220152413209,
 'contributed': 0.0004233700254022015,
 'resilient': 0.0004233700254022015,
 'increasing': 0.000846740050804403,
 'progrowth': 0.000846740050804403,

# Applying the Classifier
In this problem, the posterior probability of a tweet, represented as a set of words $(x_1, \dots, x_n)$, being written by Hillary is defined with Bayes' rule as: $$\mathbb{P}(H|x_1, \dots, x_n) \approx \frac{\mathbb{P}(H)\prod\limits_{i=1}^n \mathbb{P}(x_i | H)}{\mathbb{P}(H)\prod\limits_{i=1}^n \mathbb{P}(x_i | H) + \mathbb{P}(T)\prod\limits_{i=1}^n \mathbb{P}(x_i | T)}$$

We now have all the tools we need to calculate this posterior probability of a tweet being written by Hillary Clinton or Donald Trump. We'll walk through what everything represents step by step.

$\mathbb{P}(H)$ is the prior probability of a tweet being written by Hillary, which is calculated with $\frac{\# of Hillary tweets}{\# of total tweets}$. This information is in the `total_tweet_count` dictionary. Similarly, we can calculate $\mathbb{P}(T)$, the prior probability of a tweet being written by Trump, which is $\frac{\# of Trump tweets}{\# of total tweets}$.

$\prod\limits_{i=1}^n \mathbb{P}(x_i | H)$ is the product of the conditional probabilities for each word $\mathbb{P}(x_i | H)$. We've calculated $\mathbb{P}(x_i | H)$ for each word in the dictionary `word_probabilities_per_candidate`, so we can simply multiply them together to get their product. We take a similar approach with $\prod\limits_{i=1}^n \mathbb{P}(x_i | T)$.


## Log-Probabilities
Since we're multiplying many small values together, we run the risk of floating-point underflow. We solve this, in short, by turning $a \times b$ into $log(a) + log(b)$. Since converting the probabilities to log values does nothing to their ordering (if $\mathbb{P}(A) > \mathbb{P}(B)$, then $log(\mathbb{P}(A)) > log(\mathbb{P}(B)$)). In this way, we avoid arithmetic underflow while keeping the well-ordering of the probabilities, which is all we need for classification.

In [12]:
# create a new counter to keep track of how many we classified correctly
# and incorrectly. tuples are arranged (our guess, correct label)
outcomes = {("trump", "trump"): 0.0,
            ("trump", "hillary"): 0.0,
            ("hillary", "hillary"): 0.0,
            ("hillary", "trump"): 0.0
           }

# calculate our priors and take the log
prior_trump = total_tweet_count['trump'] / sum(total_tweet_count.values())
prior_hillary = total_tweet_count['hillary'] / sum(total_tweet_count.values())
log_prior_trump = log(prior_trump)
log_prior_hillary = log(prior_hillary)

# open the test data and test labels and refer to them as "test_data" and "test_labels", respectively
with open(TEST_DATA_PATH, 'r') as test_data, open(TEST_LABELS_PATH, 'r') as test_labels:
    # zip `test_data` and `test_labels` together to create a list of tuples of (tweet,label),
    # which we then iterate over. the `tweet` variable refers to an individual tweet
    # and the `label` variable refers to an individual label
    for test_tweet, test_label in zip(test_data, test_labels):
        # turn the test tweet into tokens
        test_tokens = tokenize(test_tweet)
        log_p_tweet_given_trump = 0.0
        log_p_tweet_given_hillary = 0.0
        for token in test_tokens:
            if token in vocabulary:
                p_word_given_trump = word_probabilities_per_candidate['trump'][token]
                log_p_word_given_trump = log(p_word_given_trump)
                # remember adding two logs is like multiplying their raw values
                log_p_tweet_given_trump += log_p_word_given_trump

                p_word_given_hillary = word_probabilities_per_candidate['hillary'][token]
                log_p_word_given_hillary = log(p_word_given_hillary)
                log_p_tweet_given_hillary += log_p_word_given_hillary
            else:
                # note that the token isn't in word_probabilities (and thus 
                # isn't seen in our train set), then we just ignore it
                # this works fine for this model, but there are a variety of NLP techniques
                # one could apply to handle "unknown tokens". Feel 
                # free to ask if you want details!
                pass
            
        # note that we don't actually have to do this, but it's here for didactic purposes
        # we can directly compare `log_p_tweet_given_hillary` with `log_p_tweet_given_trump`
        # to figure out which posterior probability will be greater!
        p_hillary_given_tweet_denominator = ((log_prior_hillary+log_p_tweet_given_hillary) + 
                                             (log_prior_trump+log_p_tweet_given_trump))
        # division is subtraction in logspace
        p_hillary_given_tweet = (log_prior_hillary+log_p_tweet_given_hillary) - p_hillary_given_tweet_denominator
        
        p_trump_given_tweet_denominator = ((log_prior_hillary+log_p_tweet_given_hillary) + 
                                           (log_prior_trump+log_p_tweet_given_trump))
        # division is subtraction in logspace
        p_trump_given_tweet = (log_prior_trump+log_p_tweet_given_trump) - p_trump_given_tweet_denominator
        
        # echoing what was said above, we could simply compare log_prior_trump+log_p_tweet_given_trump with
        # log_prior_hillary+log_p_tweet_given_hillary, since notice that the values of the denominators
        # in both cases for the full posterior probability is the same! thus, if you want to 
        # merely find out which is larger, you can just compare the numerators.
        if p_trump_given_tweet >= p_hillary_given_tweet:
            # probability that trump wrote the tweet is higher, 
            # so we predict trump. Write our prediction
            # and the correct label to the dictionary
            outcomes[("trump", LABEL_MAP[test_label])] += 1
        else:
            # probability that trump wrote the tweet is higher, 
            # so we predict trump. Write our prediction
            # and the correct label to the dictionary
            outcomes[('hillary', LABEL_MAP[test_label])] += 1

In [13]:
outcomes

{('hillary', 'hillary'): 755.0,
 ('hillary', 'trump'): 142.0,
 ('trump', 'hillary'): 44.0,
 ('trump', 'trump'): 648.0}

# Analyzing `Outcomes`
We see that we predicted "hillary" correctly 755 times, and predicted "trump" correctly 648 times. However, we made a fair amount of errors as well; confusing "trump" for "hillary" 142 times and vice versa 44 times.

Let's calculate our accuracy

In [14]:
accuracy = (outcomes[("hillary", "hillary")] + outcomes[("trump", "trump")]) / sum(outcomes.values())

print("accuracy: {}".format(accuracy))

accuracy: 0.882945248584
